# pip install

In [1]:
pip install tflearn

     |████████████████████████████████| 112kB 11.4MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127301 sha256=d6f9d9219a166c11de826264f529e1dea708af0986457f45a3187f8dd96a8e8b
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [2]:
ls 'drive/MyDrive/AIHealthcare/facial_expression_recognition_using_cnn/fer2013.csv'

drive/MyDrive/AIHealthcare/facial_expression_recognition_using_cnn/fer2013.csv


# Libraries

In [3]:
import numpy as np
import pandas as pd
import os
import argparse
import errno
import scipy.misc
import dlib
import cv2
from skimage.feature import hog
from tqdm.notebook import tqdm

In [4]:
def get_landmarks(image, rects):
    if len(rects) > 1:
        raise BaseException("TooManyFaces")
    if len(rects) == 0:
        raise BaseException("NoFaces")
    return np.matrix([[p.x, p.y] for p in predictor(image, rects[0]).parts()])

def get_new_label(label, one_hot_encoding=False):
    if one_hot_encoding:
        new_label = new_labels.index(label)
        label = list(np.zeros(len(new_labels), 'uint8'))
        label[new_label] = 1
        return label
    else:
        return new_labels.index(label)

def sliding_hog_windows(image):
    hog_windows = []
    for y in range(0, image_height, window_step):
        for x in range(0, image_width, window_step):
            window = image[y:y+window_size, x:x+window_size]
            hog_windows.extend(hog(window, orientations=8, pixels_per_cell=(8, 8),
                                            cells_per_block=(1, 1), visualize=False))
    return hog_windows

# Only GET_LANDMARKS

In [5]:
# initialization
image_height = 48
image_width = 48
window_size = 24
window_step = 6

ONE_HOT_ENCODING = True
SAVE_IMAGES = False
GET_LANDMARKS = True
GET_HOG_FEATURES = False
GET_HOG_WINDOWS_FEATURES = False
GET_HOG_IMAGES = False

SELECTED_LABELS = []
IMAGES_PER_LABEL = 500
OUTPUT_FOLDER_NAME = "Onlylandmark"
expressions = "0,1,2,3,4,5,6"

In [6]:
if expressions != "":
    expressions  = expressions.split(",")
    for i in range(0,len(expressions)):
        label = int(expressions[i])
        if (label >=0 and label<=6 ):
            SELECTED_LABELS.append(label)
if SELECTED_LABELS == []:
    SELECTED_LABELS = [0,1,2,3,4,5,6]
print( str(len(SELECTED_LABELS)) + " expressions")

7 expressions


In [7]:
# loading Dlib predictor and preparing arrays:
print( "preparing")
predictor = dlib.shape_predictor('drive/MyDrive/AIHealthcare/facial_expression_recognition_using_cnn/shape_predictor_68_face_landmarks.dat')
original_labels = [0, 1, 2, 3, 4, 5, 6]
new_labels = list(set(original_labels) & set(SELECTED_LABELS))
nb_images_per_label = list(np.zeros(len(new_labels), 'uint8'))
try:
    os.makedirs(OUTPUT_FOLDER_NAME)
    print(os.makedirs(OUTPUT_FOLDER_NAME))
except OSError as e:
    if e.errno == errno.EEXIST and os.path.isdir(OUTPUT_FOLDER_NAME):
        pass
    else:
        raise

preparing


In [8]:
print( "importing csv file")
data = pd.read_csv('drive/MyDrive/AIHealthcare/facial_expression_recognition_using_cnn/fer2013.csv')
for category in data['Usage'].unique():
    print( "converting set: " + category + "...")
    # create folder
    if not os.path.exists(category):
        try:
            os.makedirs(OUTPUT_FOLDER_NAME + '/' + category)
            print('OK')
        except OSError as e:
            if e.errno == errno.EEXIST and os.path.isdir(OUTPUT_FOLDER_NAME):
               pass
            else:
                raise

    # get samples and labels of the actual category
    category_data = data[data['Usage'] == category]
    samples = category_data['pixels'].values
    labels = category_data['emotion'].values
    
    # get images and extract features
    images = []
    labels_list = []
    landmarks = []
    hog_features = []
    hog_images = []
    for i in tqdm(range(len(samples))):
        try:
            if labels[i] in SELECTED_LABELS and nb_images_per_label[get_new_label(labels[i])] < IMAGES_PER_LABEL:
                image = np.fromstring(samples[i], dtype=int, sep=" ").reshape((image_height, image_width))
                images.append(image)
                if SAVE_IMAGES:
                    cv2.imwrite(category + '/' + str(i) + '.jpg', image)
                if GET_HOG_WINDOWS_FEATURES:
                    features = sliding_hog_windows(image)
                    f, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                                            cells_per_block=(1, 1), visualise=True)
                    hog_features.append(features)
                    if GET_HOG_IMAGES:
                        hog_images.append(hog_image)
                elif GET_HOG_FEATURES:
                    features, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                                            cells_per_block=(1, 1), visualise=True)
                    hog_features.append(features)
                    if GET_HOG_IMAGES:
                        hog_images.append(hog_image)
                if GET_LANDMARKS:
                    cv2.imwrite('temp.jpg', image)
                    image2 = cv2.imread('temp.jpg')
                    face_rects = [dlib.rectangle(left=1, top=1, right=47, bottom=47)]
                    face_landmarks = get_landmarks(image2, face_rects)
                    landmarks.append(face_landmarks)            
                labels_list.append(get_new_label(labels[i], one_hot_encoding=ONE_HOT_ENCODING))
                nb_images_per_label[get_new_label(labels[i])] += 1
        except Exception as e:
            print( "error in image: " + str(i) + " - " + str(e))

    np.save(OUTPUT_FOLDER_NAME + '/' + category + '/images.npy', images)
    if ONE_HOT_ENCODING:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/labels.npy', labels_list)
    else:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/labels.npy', labels_list)
    if GET_LANDMARKS:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/landmarks.npy', landmarks)
    if GET_HOG_FEATURES or GET_HOG_WINDOWS_FEATURES:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/hog_features.npy', hog_features)
        if GET_HOG_IMAGES:
            np.save(OUTPUT_FOLDER_NAME + '/' + category + '/hog_images.npy', hog_images)

importing csv file
converting set: Training...
OK



converting set: PublicTest...
OK



converting set: PrivateTest...
OK


# LANDMARKS + HOG


In [9]:
# initialization
image_height = 48
image_width = 48
window_size = 24
window_step = 6

ONE_HOT_ENCODING = True
SAVE_IMAGES = False
GET_LANDMARKS = True
GET_HOG_FEATURES = True
GET_HOG_WINDOWS_FEATURES = False
GET_HOG_IMAGES = False

SELECTED_LABELS = []
IMAGES_PER_LABEL = 500
OUTPUT_FOLDER_NAME = "LANDMARKS_HOG"
expressions = "0,1,2,3,4,5,6"

In [10]:
if expressions != "":
    expressions  = expressions.split(",")
    for i in range(0,len(expressions)):
        label = int(expressions[i])
        if (label >=0 and label<=6 ):
            SELECTED_LABELS.append(label)
if SELECTED_LABELS == []:
    SELECTED_LABELS = [0,1,2,3,4,5,6]
print( str(len(SELECTED_LABELS)) + " expressions")

7 expressions


In [11]:
# loading Dlib predictor and preparing arrays:
print( "preparing")
predictor = dlib.shape_predictor('drive/MyDrive/AIHealthcare/facial_expression_recognition_using_cnn/shape_predictor_68_face_landmarks.dat')
original_labels = [0, 1, 2, 3, 4, 5, 6]
new_labels = list(set(original_labels) & set(SELECTED_LABELS))
nb_images_per_label = list(np.zeros(len(new_labels), 'uint8'))
try:
    os.makedirs(OUTPUT_FOLDER_NAME)
    print(os.makedirs(OUTPUT_FOLDER_NAME))
except OSError as e:
    if e.errno == errno.EEXIST and os.path.isdir(OUTPUT_FOLDER_NAME):
        pass
    else:
        raise

preparing


In [12]:
print( "importing csv file")
data = pd.read_csv('drive/MyDrive/AIHealthcare/facial_expression_recognition_using_cnn/fer2013.csv')
for category in data['Usage'].unique():
    print( "converting set: " + category + "...")
    # create folder
    if not os.path.exists(category):
        try:
            os.makedirs(OUTPUT_FOLDER_NAME + '/' + category)
            print('OK')
        except OSError as e:
            if e.errno == errno.EEXIST and os.path.isdir(OUTPUT_FOLDER_NAME):
               pass
            else:
                raise

    # get samples and labels of the actual category
    category_data = data[data['Usage'] == category]
    samples = category_data['pixels'].values
    labels = category_data['emotion'].values
    
    # get images and extract features
    images = []
    labels_list = []
    landmarks = []
    hog_features = []
    hog_images = []
    for i in tqdm(range(len(samples))):
        try:
            if labels[i] in SELECTED_LABELS and nb_images_per_label[get_new_label(labels[i])] < IMAGES_PER_LABEL:
                image = np.fromstring(samples[i], dtype=int, sep=" ").reshape((image_height, image_width))
                images.append(image)
                if SAVE_IMAGES:
                    cv2.imwrite(category + '/' + str(i) + '.jpg', image)
                if GET_HOG_WINDOWS_FEATURES:
                    features = sliding_hog_windows(image)
                    f, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                                            cells_per_block=(1, 1), visualize=True)
                    hog_features.append(features)
                    if GET_HOG_IMAGES:
                        hog_images.append(hog_image)
                elif GET_HOG_FEATURES:
                    features, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                                            cells_per_block=(1, 1), visualize=True)
                    hog_features.append(features)
                    if GET_HOG_IMAGES:
                        hog_images.append(hog_image)
                if GET_LANDMARKS:
                    cv2.imwrite('temp.jpg', image)
                    image2 = cv2.imread('temp.jpg')
                    face_rects = [dlib.rectangle(left=1, top=1, right=47, bottom=47)]
                    face_landmarks = get_landmarks(image2, face_rects)
                    landmarks.append(face_landmarks)            
                labels_list.append(get_new_label(labels[i], one_hot_encoding=ONE_HOT_ENCODING))
                nb_images_per_label[get_new_label(labels[i])] += 1
        except Exception as e:
            print( "error in image: " + str(i) + " - " + str(e))

    np.save(OUTPUT_FOLDER_NAME + '/' + category + '/images.npy', images)
    if ONE_HOT_ENCODING:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/labels.npy', labels_list)
    else:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/labels.npy', labels_list)
    if GET_LANDMARKS:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/landmarks.npy', landmarks)
    if GET_HOG_FEATURES or GET_HOG_WINDOWS_FEATURES:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/hog_features.npy', hog_features)
        if GET_HOG_IMAGES:
            np.save(OUTPUT_FOLDER_NAME + '/' + category + '/hog_images.npy', hog_images)

importing csv file
converting set: Training...
OK



converting set: PublicTest...
OK



converting set: PrivateTest...
OK


# Face landmarks + HOG + sliding window

In [13]:
# initialization
image_height = 48
image_width = 48
window_size = 24
window_step = 6

ONE_HOT_ENCODING = True
SAVE_IMAGES = False
GET_LANDMARKS = True
GET_HOG_FEATURES = True
GET_HOG_WINDOWS_FEATURES = True
GET_HOG_IMAGES = False

SELECTED_LABELS = []
IMAGES_PER_LABEL = 500
OUTPUT_FOLDER_NAME = "Facelandmarks_HOG_sliding window"
expressions = "0,1,2,3,4,5,6"

In [14]:
if expressions != "":
    expressions  = expressions.split(",")
    for i in range(0,len(expressions)):
        label = int(expressions[i])
        if (label >=0 and label<=6 ):
            SELECTED_LABELS.append(label)
if SELECTED_LABELS == []:
    SELECTED_LABELS = [0,1,2,3,4,5,6]
print( str(len(SELECTED_LABELS)) + " expressions")

7 expressions


In [15]:
# loading Dlib predictor and preparing arrays:
print( "preparing")
predictor = dlib.shape_predictor('drive/MyDrive/AIHealthcare/facial_expression_recognition_using_cnn/shape_predictor_68_face_landmarks.dat')
original_labels = [0, 1, 2, 3, 4, 5, 6]
new_labels = list(set(original_labels) & set(SELECTED_LABELS))
nb_images_per_label = list(np.zeros(len(new_labels), 'uint8'))
try:
    os.makedirs(OUTPUT_FOLDER_NAME)
    print(os.makedirs(OUTPUT_FOLDER_NAME))
except OSError as e:
    if e.errno == errno.EEXIST and os.path.isdir(OUTPUT_FOLDER_NAME):
        pass
    else:
        raise

preparing


In [16]:
print( "importing csv file")
data = pd.read_csv('drive/MyDrive/AIHealthcare/facial_expression_recognition_using_cnn/fer2013.csv')
for category in data['Usage'].unique():
    print( "converting set: " + category + "...")
    # create folder
    if not os.path.exists(category):
        try:
            os.makedirs(OUTPUT_FOLDER_NAME + '/' + category)
            print('OK')
        except OSError as e:
            if e.errno == errno.EEXIST and os.path.isdir(OUTPUT_FOLDER_NAME):
               pass
            else:
                raise

    # get samples and labels of the actual category
    category_data = data[data['Usage'] == category]
    samples = category_data['pixels'].values
    labels = category_data['emotion'].values
    
    # get images and extract features
    images = []
    labels_list = []
    landmarks = []
    hog_features = []
    hog_images = []
    for i in tqdm(range(len(samples))):
        try:
            if labels[i] in SELECTED_LABELS and nb_images_per_label[get_new_label(labels[i])] < IMAGES_PER_LABEL:
                image = np.fromstring(samples[i], dtype=int, sep=" ").reshape((image_height, image_width))
                images.append(image)
                if SAVE_IMAGES:
                    cv2.imwrite(category + '/' + str(i) + '.jpg', image)
                if GET_HOG_WINDOWS_FEATURES:
                    features = sliding_hog_windows(image)
                    f, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                                            cells_per_block=(1, 1), visualize=True)
                    hog_features.append(features)
                    if GET_HOG_IMAGES:
                        hog_images.append(hog_image)
                elif GET_HOG_FEATURES:
                    features, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                                            cells_per_block=(1, 1), visualize=True)
                    hog_features.append(features)
                    if GET_HOG_IMAGES:
                        hog_images.append(hog_image)
                if GET_LANDMARKS:
                    cv2.imwrite('temp.jpg', image)
                    image2 = cv2.imread('temp.jpg')
                    face_rects = [dlib.rectangle(left=1, top=1, right=47, bottom=47)]
                    face_landmarks = get_landmarks(image2, face_rects)
                    landmarks.append(face_landmarks)            
                labels_list.append(get_new_label(labels[i], one_hot_encoding=ONE_HOT_ENCODING))
                nb_images_per_label[get_new_label(labels[i])] += 1
        except Exception as e:
            print( "error in image: " + str(i) + " - " + str(e))

    np.save(OUTPUT_FOLDER_NAME + '/' + category + '/images.npy', images)
    if ONE_HOT_ENCODING:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/labels.npy', labels_list)
    else:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/labels.npy', labels_list)
    if GET_LANDMARKS:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/landmarks.npy', landmarks)
    if GET_HOG_FEATURES or GET_HOG_WINDOWS_FEATURES:
        np.save(OUTPUT_FOLDER_NAME + '/' + category + '/hog_features.npy', hog_features)
        if GET_HOG_IMAGES:
            np.save(OUTPUT_FOLDER_NAME + '/' + category + '/hog_images.npy', hog_images)

importing csv file
converting set: Training...
OK



converting set: PublicTest...
OK



converting set: PrivateTest...
OK


# Zip

In [17]:
!zip -r "/content/Onlylandmark.zip" "/content/Onlylandmark"

  adding: content/Onlylandmark/ (stored 0%)
  adding: content/Onlylandmark/Training/ (stored 0%)
  adding: content/Onlylandmark/Training/landmarks.npy (deflated 87%)
  adding: content/Onlylandmark/Training/images.npy (deflated 82%)
  adding: content/Onlylandmark/Training/labels.npy (deflated 98%)
  adding: content/Onlylandmark/PublicTest/ (stored 0%)
  adding: content/Onlylandmark/PublicTest/landmarks.npy (deflated 87%)
  adding: content/Onlylandmark/PublicTest/images.npy (deflated 82%)
  adding: content/Onlylandmark/PublicTest/labels.npy (deflated 97%)
  adding: content/Onlylandmark/PrivateTest/ (stored 0%)
  adding: content/Onlylandmark/PrivateTest/landmarks.npy (deflated 83%)
  adding: content/Onlylandmark/PrivateTest/images.npy (deflated 82%)
  adding: content/Onlylandmark/PrivateTest/labels.npy (deflated 86%)


In [18]:
!zip -r "/content/LANDMARKS_HOG.zip" "/content/LANDMARKS_HOG"

  adding: content/LANDMARKS_HOG/ (stored 0%)
  adding: content/LANDMARKS_HOG/Training/ (stored 0%)
  adding: content/LANDMARKS_HOG/Training/landmarks.npy (deflated 87%)
  adding: content/LANDMARKS_HOG/Training/hog_features.npy (deflated 52%)
  adding: content/LANDMARKS_HOG/Training/images.npy (deflated 82%)
  adding: content/LANDMARKS_HOG/Training/labels.npy (deflated 98%)
  adding: content/LANDMARKS_HOG/PublicTest/ (stored 0%)
  adding: content/LANDMARKS_HOG/PublicTest/landmarks.npy (deflated 87%)
  adding: content/LANDMARKS_HOG/PublicTest/hog_features.npy (deflated 54%)
  adding: content/LANDMARKS_HOG/PublicTest/images.npy (deflated 82%)
  adding: content/LANDMARKS_HOG/PublicTest/labels.npy (deflated 97%)
  adding: content/LANDMARKS_HOG/PrivateTest/ (stored 0%)
  adding: content/LANDMARKS_HOG/PrivateTest/landmarks.npy (deflated 83%)
  adding: content/LANDMARKS_HOG/PrivateTest/hog_features.npy (deflated 51%)
  adding: content/LANDMARKS_HOG/PrivateTest/images.npy (deflated 82%)
  addin

In [19]:
!zip -r "/content/Facelandmarks_HOG_sliding_window.zip" "/content/Facelandmarks_HOG_sliding_window"

  adding: content/Facelandmarks_HOG_sliding_window/ (stored 0%)
  adding: content/Facelandmarks_HOG_sliding_window/Training/ (stored 0%)
  adding: content/Facelandmarks_HOG_sliding_window/Training/landmarks.npy (deflated 87%)
  adding: content/Facelandmarks_HOG_sliding_window/Training/hog_features.npy (deflated 41%)
  adding: content/Facelandmarks_HOG_sliding_window/Training/images.npy (deflated 82%)
  adding: content/Facelandmarks_HOG_sliding_window/Training/labels.npy (deflated 98%)
  adding: content/Facelandmarks_HOG_sliding_window/PublicTest/ (stored 0%)
  adding: content/Facelandmarks_HOG_sliding_window/PublicTest/landmarks.npy (deflated 87%)
  adding: content/Facelandmarks_HOG_sliding_window/PublicTest/hog_features.npy (deflated 41%)
  adding: content/Facelandmarks_HOG_sliding_window/PublicTest/images.npy (deflated 82%)
  adding: content/Facelandmarks_HOG_sliding_window/PublicTest/labels.npy (deflated 97%)
  adding: content/Facelandmarks_HOG_sliding_window/PrivateTest/ (stored 0%